<a href="https://colab.research.google.com/github/lingyixu/CS523-Deep-Learning/blob/main/Non_Graph_NN/MLP_Multiclass_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load packages

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.0+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
# import torch_geometric
# from torch_geometric.data import Dataset, Data
# from torch_geometric.nn import GCNConv, GATConv
# from torch_geometric.transforms import RandomNodeSplit

In [3]:
import pandas as pd
import numpy as np
import json
import glob
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [4]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/Shareddrives/CS523/lastFM-data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load dataset: LastFM

In [5]:
df_feat = pd.read_json(drive_path + 'processed_feature.json')   # load features
arr_feat = df_feat.T.values
df_edge = pd.read_csv(drive_path + 'lastfm_asia_edges.csv')   # load edge indices
arr_edge = df_edge.T.values
df_tar = pd.read_csv(drive_path + 'lastfm_asia_target.csv', index_col=0)   # load targets
arr_tar = df_tar.values

num_nodes = arr_feat.shape[0]
num_features = arr_feat.shape[1]
num_classes = len(df_tar.target.unique())
classes = df_tar.target.sort_values().unique()

print('Dataset:')
print('====================')
print(f'Number of users: {num_nodes}')
print(f'Number of features: {num_features}')
# print(f'Number of edges: {arr_edge.shape[1]}')
print(f'Number of distinct regions: {num_classes}')
print(f'All region classes: {classes}')

Dataset:
Number of users: 7624
Number of features: 7842
Number of distinct regions: 18
All region classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


### Create Dataset

In [6]:
class LastFM(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.transform = transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        data = self.x[index,]
        if self.transform is not None:
            data = self.transform(data)
        if self.y is not None:
            return data, self.y[index]
        else:
            return data

In [7]:
# fm_dataset = LastFM(x=arr_feat, y=arr_tar)
x_train, x_test, y_train, y_test = train_test_split(arr_feat, arr_tar, test_size=0.2, random_state=42)
train_set = LastFM(x=x_train, y=y_train)
test_set = LastFM(x=x_test, y=y_test)

In [8]:
print(f'Number of training data: {len(train_set)}')
print(f'Number of testing data: {len(test_set)}')

Number of training data: 6099
Number of testing data: 1525


### Build Model

In [9]:
# ref: https://www.kaggle.com/code/pinocookie/pytorch-simple-mlp/notebook
class MLP(torch.nn.Module):
    def __init__(self, hidden_channels1, hidden_channels2):
        super().__init__()
        # torch.manual_seed(42)
        self.lin1 = nn.Linear(num_features, hidden_channels1)
        self.lin2 = nn.Linear(hidden_channels1, hidden_channels2)
        self.lin3 = nn.Linear(hidden_channels2, num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.1)
        x = self.lin2(x)
        x = x.relu()
        x = F.dropout(x, p=0.1)
        x = self.lin3(x)
        return x

In [10]:
model = MLP(256, 128)
print(model)

MLP(
  (lin1): Linear(in_features=7842, out_features=256, bias=True)
  (lin2): Linear(in_features=256, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=18, bias=True)
)


In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=5e-4)

In [12]:
def train():
      model.train()
      optimizer.zero_grad()
      out = model(train_set.x)
      loss = criterion(out, train_set.y.flatten())
      loss.backward()
      optimizer.step()
      return loss

def test():
      model.eval()
      out = model(test_set.x)
      pred = out.argmax(dim=1)
      test_correct = pred == test_set.y.flatten()
      test_acc = int(test_correct.sum()) / len(test_set)
      return test_acc

for epoch in range(100):
    loss = train()
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

Epoch: 01, Loss: 2.9106
Epoch: 02, Loss: 2.8380
Epoch: 03, Loss: 2.7612
Epoch: 04, Loss: 2.6711
Epoch: 05, Loss: 2.5722
Epoch: 06, Loss: 2.4667
Epoch: 07, Loss: 2.3637
Epoch: 08, Loss: 2.2737
Epoch: 09, Loss: 2.1928
Epoch: 10, Loss: 2.1194
Epoch: 11, Loss: 2.0496
Epoch: 12, Loss: 1.9867
Epoch: 13, Loss: 1.9186
Epoch: 14, Loss: 1.8610
Epoch: 15, Loss: 1.7978
Epoch: 16, Loss: 1.7507
Epoch: 17, Loss: 1.6949
Epoch: 18, Loss: 1.6462
Epoch: 19, Loss: 1.6009
Epoch: 20, Loss: 1.5547
Epoch: 21, Loss: 1.5069
Epoch: 22, Loss: 1.4615
Epoch: 23, Loss: 1.4243
Epoch: 24, Loss: 1.3831
Epoch: 25, Loss: 1.3472
Epoch: 26, Loss: 1.3057
Epoch: 27, Loss: 1.2725
Epoch: 28, Loss: 1.2355
Epoch: 29, Loss: 1.2026
Epoch: 30, Loss: 1.1688
Epoch: 31, Loss: 1.1352
Epoch: 32, Loss: 1.1092
Epoch: 33, Loss: 1.0787
Epoch: 34, Loss: 1.0486
Epoch: 35, Loss: 1.0124
Epoch: 36, Loss: 0.9949
Epoch: 37, Loss: 0.9621
Epoch: 38, Loss: 0.9333
Epoch: 39, Loss: 0.9107
Epoch: 40, Loss: 0.8850
Epoch: 41, Loss: 0.8584
Epoch: 42, Loss:

In [13]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7633
